In [1]:
import duckdb

# puzzle input goes here
input = """
"""

In [2]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), hands AS (
        SELECT
            GENERATE_SUBSCRIPTS(STRING_SPLIT(input, '\n'), 1) hand_no,
            UNNEST(STRING_SPLIT(input, '\n')) i,
            CAST(STRING_SPLIT(i, ' ')[2] AS INT) bid,
            STRING_SPLIT(STRING_SPLIT(i, ' ')[1], '') hand
        FROM input
    ), card_values AS (
        SELECT
            UNNEST(ARRAY['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']) card,
            UNNEST(ARRAY['14', '13', '12', '11', '10', '09', '08', '07', '06', '05', '04', '03', '02']) v
    ), cards AS (
        SELECT
            hand_no,
            GENERATE_SUBSCRIPTS(hand, 1) card_no,
            UNNEST(hand) card
        FROM hands
    ), occurrences AS (
        SELECT hand_no, ARRAY_SORT(ARRAY_AGG(n)) n
        FROM (
            SELECT hand_no, card, COUNT(*) n
            FROM cards
            GROUP BY hand_no, card
        )
        GROUP BY hand_no
    ), untie AS (
        SELECT hand_no, STRING_AGG(v, '' ORDER BY card_no) untie
        FROM cards, card_values
        WHERE cards.card = card_values.card
        GROUP BY hand_no
    ), strength AS (
        SELECT
            hand_no,
            n[-1] * 10 + COALESCE(n[-2], 0) strength
        FROM occurrences
    ), hand_rank AS (
        SELECT h.hand_no, h.bid, h.hand, RANK() OVER w hand_rank
        FROM hands h, strength s, untie u
        WHERE h.hand_no = s.hand_no
        AND s.hand_no = u.hand_no
        WINDOW w AS (ORDER BY s.strength, u.untie)
    )
    SELECT SUM(bid * hand_rank) ans
    FROM hand_rank
""" % input)

┌───────────┐
│    ans    │
│  int128   │
├───────────┤
│ 251106089 │
└───────────┘

In [3]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), hands AS (
        SELECT
            GENERATE_SUBSCRIPTS(STRING_SPLIT(input, '\n'), 1) hand_no,
            UNNEST(STRING_SPLIT(input, '\n')) i,
            CAST(STRING_SPLIT(i, ' ')[2] AS INT) bid,
            STRING_SPLIT(STRING_SPLIT(i, ' ')[1], '') hand
        FROM input
    ), card_values AS (
        SELECT
            UNNEST(ARRAY['A', 'K', 'Q', 'T', '9', '8', '7', '6', '5', '4', '3', '2', 'J']) card,
            UNNEST(ARRAY['14', '13', '12', '10', '09', '08', '07', '06', '05', '04', '03', '02', '00']) v
    ), cards AS (
        SELECT
            hand_no,
            GENERATE_SUBSCRIPTS(hand, 1) card_no,
            UNNEST(hand) card
        FROM hands
    ), occurrences AS (
        SELECT hand_no, ARRAY_SORT(ARRAY_AGG(n) FILTER(card <> 'J')) n, ANY_VALUE(n) FILTER(card = 'J') n_J
        FROM (
            SELECT hand_no, card, COUNT(*) n
            FROM cards
            GROUP BY hand_no, card
        )
        GROUP BY hand_no
    ), untie AS (
        SELECT hand_no, STRING_AGG(v, '' ORDER BY card_no) untie
        FROM cards, card_values
        WHERE cards.card = card_values.card
        GROUP BY hand_no
    ), strength AS (
        SELECT
            hand_no,
            (COALESCE(n[-1], 0) + COALESCE(n_J, 0)) * 10 + COALESCE(n[-2], 0) strength
        FROM occurrences
    ), hand_rank AS (
        SELECT h.hand_no, h.bid, h.hand, RANK() OVER w hand_rank, s.strength, u.untie
        FROM hands h, strength s, untie u
        WHERE h.hand_no = s.hand_no
        AND s.hand_no = u.hand_no
        WINDOW w AS (ORDER BY s.strength, u.untie)
    )
    SELECT SUM(bid * hand_rank) ans
    FROM hand_rank
""" % input)

┌───────────┐
│    ans    │
│  int128   │
├───────────┤
│ 249620106 │
└───────────┘